In [1]:
from DIAMOnD.DIAMOnD import *
from network_separation_simplified.separation import *
from toolbox_simplified.wrappers import *


# convert drug genes: 将化合物-基因一对一的格式改为一个化合物对多个基因，并将其用｜｜分开

In [ ]:
data = pd.read_excel('/Users/yanlixu/Desktop/pingan/合作/CDK12/network_based/related_datasets_20220619.xlsx', sheet_name = 'dti_from_chenfeixiong')
data = data.applymap(str)
data['Drug_Target (Gene Eentrez IDs)'] = data.groupby(['DrugID (DrugBank ID)'])['Drug_Target (Gene Eentrez IDs)'].transform(lambda x: '||'.join(x))
data = data.drop_duplicates()
data

In [ ]:
data.to_csv('dti_from_chenfeixiong_compress.csv')

# get diamond genes： 从100个seed gene, 运行Diamond得到另外100个基因

In [ ]:
network_edgelist_file = '/Users/yanlixu/Desktop/git_code/drug_repurposing_202206/network_based_2/DIAMOnD/Example/ppi_chenfeixiong.txt'
seeds_file = '/Users/yanlixu/Desktop/git_code/drug_repurposing_202206/network_based_2/DIAMOnD/Example/breast_cancer_seed_genes.txt'
max_number_of_added_nodes = 100
alpha = 1
outfile_name = 'brease_cancer_diamond_genes_' + str(max_number_of_added_nodes) + '.txt'

G_original, seed_genes = read_input(network_edgelist_file, seeds_file)
added_nodes = DIAMOnD(G_original, seed_genes, max_number_of_added_nodes, alpha, outfile=outfile_name)
print("\n results have been saved to '%s' \n" % outfile_name)






# 输入：（1）gwas及Diamond基因；（2）基因-化合物对应关系
# 结果：化合物及其对应的基因

In [ ]:
gwas_diamond_genes = pd.read_excel('/Users/yanlixu/Desktop/pingan/合作/CDK12/network_based/1_her2_breast_cancer_20220620.xlsx', sheet_name = '4_gwas_diamond_100')
drug_target_interaction = pd.read_excel('/Users/yanlixu/Desktop/pingan/合作/CDK12/network_based/related_datasets_20220619.xlsx', sheet_name = 'dti_from_chenfeixiong')

gwas_diamond_gene_drugs = pd.merge(gwas_diamond_genes, drug_target_interaction, left_on='gene_id', right_on='Drug_Target (Gene Eentrez IDs)')
gwas_diamond_gene_drugs.to_csv('gwas_diamond_100_drug.csv')


In [ ]:
gwas_diamond_gene_drugs = gwas_diamond_gene_drugs.applymap(str)
gwas_diamond_gene_drugs['Drug_Target (Gene Eentrez IDs)'] = gwas_diamond_gene_drugs.groupby(['DrugID (DrugBank ID)'])['Drug_Target (Gene Eentrez IDs)'].transform(lambda x: '||'.join(x))
gwas_diamond_gene_drugs_sub = gwas_diamond_gene_drugs[['DrugID (DrugBank ID)', 'Drug_Target (Gene Eentrez IDs)']]
gwas_diamond_gene_drugs_sub = gwas_diamond_gene_drugs_sub.drop_duplicates()
gwas_diamond_gene_drugs_sub


In [ ]:
gwas_diamond_gene_drugs_sub.to_csv('gwas_diamond_100_dti.csv')

# z value: drug targets and breast cancer gene modules proximity
# gene module: 4_gwas_diamond_100
# drug targets: 5_drug_target_w_diamond_100_cmp

In [ ]:
network_file = "/Users/yanlixu/Desktop/git_code/drug_repurposing_202206/network_based_2/toolbox_simplified/ppi_chenfeixiong_w_edge.sif.txt"
her2_file = '/Users/yanlixu/Desktop/pingan/合作/CDK12/network_based/1_her2_breast_cancer_20220620.xlsx'
#drug_genes_file = '/Users/yanlixu/Desktop/pingan/合作/CDK12/network_based/related_datasets_20220619.xlsx'

network = get_network(network_file, only_lcc = True)

# get disease genes
her2_bc_data = pd.read_excel(her2_file, sheet_name='4_gwas_diamond_100')
her2_breast_cancer_genes = [str(gene) for gene in her2_bc_data['gene_id'].tolist()]

# get drug related genes   
drug_genes = pd.read_excel(her2_file, sheet_name='5_drug_target_w_diamond_100_cmp')

# calculate proximity of disease genes and drug genes
#fout = open('drug_breast_cancer_proximity.txt', 'w')
#fout.write('DrugID' + '\t' + 'Drug_Target' + '\t' + 'd' + '\t' + 'z' + '\t' + 'mean' + '\t' + 'sd' + '\n')
result = []
for index, row in drug_genes.iterrows():
    drug_gene_list = str(row['Drug_Target (Gene Eentrez IDs)']).split('||')
    print(row[0])
    print(row[1])

    d, z, (mean, sd) = calculate_proximity(network, her2_breast_cancer_genes, drug_gene_list, min_bin_size = 450, seed=452456)

    print(d, z, (mean, sd))
    #fout.write(str(row[0]) + '\t' + str(row[1]) + '\t' + str(d)+ '\t' + str(z) + '\t' + str(mean) + '\t' + str(sd) + '\n')
    result.append([row[0], row[1], d, z, mean, sd])

In [ ]:
df = pd.DataFrame(result, columns = ['DrugID', 'DrugTarget', 'd', 'z', 'mean', 'sd'])
df

In [ ]:
df.to_csv('drug_genes_disease_proximity.csv')

# Sab: separation between drug targets
# drug targets: 5_drug_target_w_diamond_100_cmp

In [5]:
network_file = '/Users/yanlixu/Desktop/git_code/drug_repurposing_202206/network_based_2/toolbox_simplified/ppi_chenfeixiong.tsv'

drug_gene_file = '/Users/yanlixu/Desktop/pingan/合作/CDK12/network_based/1_her2_breast_cancer_20220620.xlsx'

# read network
G = read_network(network_file)
# get all genes ad remove self links
all_genes_in_network = set(G.nodes())
remove_self_links(G)

# read data from excel
drug_genes = pd.read_excel(drug_gene_file, sheet_name='5_drug_target_w_diamond_100_cmp')
drug_genes = drug_genes.astype(str)
# convert genes column to list
drug_genes['Drug_Target (Gene Eentrez IDs)'] = drug_genes['Drug_Target (Gene Eentrez IDs)'].apply(lambda x: x.split('||'))
# dataframe to dictionary
drug_genes.set_index('DrugID (DrugBank ID)', inplace=True)
drug_gene_dic = drug_genes.to_dict()['Drug_Target (Gene Eentrez IDs)']




> done loading network:
> network contains 15970 nodes and 217160 links


In [6]:
drug_gene_dic

{'DB00030': ['3480', '5925', '3480', '5925'],
 'DB00046': ['3480'],
 'DB00047': ['3480'],
 'DB00071': ['3480', '5925', '3480', '5925'],
 'DB01277': ['3480'],
 'DB02944': ['3480'],
 'DB04395': ['3480'],
 'DB07156': ['3480'],
 'DB07474': ['3480'],
 'DB00179': ['3480'],
 'DB02197': ['3480'],
 'DB04216': ['3480',
  '1956',
  '6714',
  '207',
  '1432',
  '3480',
  '1956',
  '6714',
  '207',
  '1432',
  '3480',
  '1956',
  '6714',
  '207',
  '1432',
  '3480',
  '1956',
  '6714',
  '207',
  '1432',
  '3480',
  '1956',
  '6714',
  '207',
  '1432'],
 'DB07159': ['3480',
  '2263',
  '1956',
  '25',
  '5599',
  '6714',
  '28996',
  '3480',
  '2263',
  '1956',
  '25',
  '5599',
  '6714',
  '28996',
  '3480',
  '2263',
  '1956',
  '25',
  '5599',
  '6714',
  '28996',
  '3480',
  '2263',
  '1956',
  '25',
  '5599',
  '6714',
  '28996',
  '3480',
  '2263',
  '1956',
  '25',
  '5599',
  '6714',
  '28996',
  '3480',
  '2263',
  '1956',
  '25',
  '5599',
  '6714',
  '28996',
  '3480',
  '2263',
  '1956'

In [7]:
fout = open('her2_drug_genes_separation_20220703.txt', 'w')
fout.write('drug_a' + '\t' + 'drug_a_genes' + '\t' + 'drug_b' + '\t' + 'drug_b_genes' + '\t' + 'dA' + '\t' + 'dB' + '\t' + 'dAB' + '\t' + 'sAB' + '\n')
for drug_a in drug_gene_dic.keys():
    drug_a_genes = drug_gene_dic[drug_a]
    for drug_b in drug_gene_dic.keys():
        drug_b_genes = drug_gene_dic[drug_b]
        result = calculate_distance_bw_2gene_sets(G, all_genes_in_network, drug_a_genes, drug_b_genes)
        fout.write(drug_a + '\t' + '||'.join(drug_gene_dic[drug_a]) + '\t' + drug_b + '\t' + '||'.join(drug_gene_dic[drug_b])+ '\t' + str(result[0]) + '\t' + str(result[1]) + '\t' + str(result[2]) + '\t' + str(result[3]) + '\n')
fout.close()

# combine drug target separation file and drug target-disease proximity file

In [86]:
# drug target-disease proximity file:

drug_proximity_file = '/Users/yanlixu/Desktop/pingan/合作/CDK12/network_based/3_drug_combination_results.xlsx'
drug_proximity = pd.read_excel(drug_targets_separation_file, sheet_name = '1_drug_breast_cancer_proximity')
drug_proximity = drug_proximity.set_index('DrugID')
drug_proximity

,DrugTarget,d,z,mean,sd
DrugID,,,,,
DB00030,3480||5925||3480||5925,1.774869,-4.091324,2.016963,0.059173
DB00046,3480,2.057592,-0.963609,2.152366,0.098354
DB00047,3480,2.057592,-0.963609,2.152366,0.098354
DB00071,3480||5925||3480||5925,1.774869,-4.091324,2.016963,0.059173
DB01277,3480,2.057592,-0.963609,2.152366,0.098354
...,...,...,...,...,...
DB07795,1432,1.905759,-1.743561,2.081424,0.100751
DB08010,1432,1.905759,-1.743561,2.081424,0.100751
DB08015,1432,1.905759,-1.743561,2.081424,0.100751


In [87]:
# drug targets and drug targets separation

drug_targets_separation_file = '/Users/yanlixu/Desktop/pingan/合作/CDK12/network_based/3_drug_combination_results.xlsx'
drug_targets_separation = pd.read_excel(drug_targets_separation_file, sheet_name = '1_drug_drug_separation')
drug_targets_separation


,drug_a,drug_a_genes,drug_b,drug_b_genes,dA,dB,dAB,sAB
0,DB00030,3480||5925||3480||5925,DB00030,3480||5925||3480||5925,2.0,2.0,0.000000,-2.000000
1,DB00030,3480||5925||3480||5925,DB00046,3480,2.0,0.0,0.666667,-0.333333
2,DB00030,3480||5925||3480||5925,DB00047,3480,2.0,0.0,0.666667,-0.333333
3,DB00030,3480||5925||3480||5925,DB00071,3480||5925||3480||5925,2.0,2.0,0.000000,-2.000000
4,DB00030,3480||5925||3480||5925,DB01277,3480,2.0,0.0,0.666667,-0.333333
...,...,...,...,...,...,...,...,...
327179,DB01029,3725,DB07795,1432,0.0,0.0,2.000000,2.000000
327180,DB01029,3725,DB08010,1432,0.0,0.0,2.000000,2.000000
327181,DB01029,3725,DB08015,1432,0.0,0.0,2.000000,2.000000
327182,DB01029,3725,DB00570,3725,0.0,0.0,0.000000,0.000000


In [88]:
# add z values(drug target-disease proximity file) to drug targets and drug targets separation file

drug_targets_separation['breast_cancer_z_a'] = drug_targets_separation['drug_a'].apply(lambda x: drug_proximity.loc[x, 'z'])
drug_targets_separation['breast_cancer_z_b'] = drug_targets_separation['drug_b'].apply(lambda x: drug_proximity.loc[x, 'z'])

drug_targets_separation



,drug_a,drug_a_genes,drug_b,drug_b_genes,dA,dB,dAB,sAB,breast_cancer_z_a,breast_cancer_z_b
0,DB00030,3480||5925||3480||5925,DB00030,3480||5925||3480||5925,2.0,2.0,0.000000,-2.000000,-4.091324,-4.091324
1,DB00030,3480||5925||3480||5925,DB00046,3480,2.0,0.0,0.666667,-0.333333,-4.091324,-0.963609
2,DB00030,3480||5925||3480||5925,DB00047,3480,2.0,0.0,0.666667,-0.333333,-4.091324,-0.963609
3,DB00030,3480||5925||3480||5925,DB00071,3480||5925||3480||5925,2.0,2.0,0.000000,-2.000000,-4.091324,-4.091324
4,DB00030,3480||5925||3480||5925,DB01277,3480,2.0,0.0,0.666667,-0.333333,-4.091324,-0.963609
...,...,...,...,...,...,...,...,...,...,...
327179,DB01029,3725,DB07795,1432,0.0,0.0,2.000000,2.000000,-3.493732,-1.743561
327180,DB01029,3725,DB08010,1432,0.0,0.0,2.000000,2.000000,-3.493732,-1.743561
327181,DB01029,3725,DB08015,1432,0.0,0.0,2.000000,2.000000,-3.493732,-1.743561
327182,DB01029,3725,DB00570,3725,0.0,0.0,0.000000,0.000000,-3.493732,-3.493732


In [89]:
# remove unwanted rows

drug_targets_separation_filtered = drug_targets_separation[(drug_targets_separation['sAB'] >= 0) & (drug_targets_separation['breast_cancer_z_a'] < 0) & (drug_targets_separation['breast_cancer_z_b'] < 0)]
drug_targets_separation_filtered
 

,drug_a,drug_a_genes,drug_b,drug_b_genes,dA,dB,dAB,sAB,breast_cancer_z_a,breast_cancer_z_b
20,DB00030,3480||5925||3480||5925,DB04147,1072,2.0,0.0,2.0,1.0,-4.091324,-1.126009
21,DB00030,3480||5925||3480||5925,DB08080,57591,2.0,0.0,2.0,1.0,-4.091324,-1.115153
22,DB00030,3480||5925||3480||5925,DB00294,5241||2099||5241||2099,2.0,1.0,1.5,0.0,-4.091324,-7.544839
23,DB00030,3480||5925||3480||5925,DB00304,5241||2099||5241||2099,2.0,1.0,1.5,0.0,-4.091324,-7.544839
24,DB00030,3480||5925||3480||5925,DB00351,5241||2908||5241||2908,2.0,2.0,2.0,0.0,-4.091324,-4.594288
...,...,...,...,...,...,...,...,...,...,...
327179,DB01029,3725,DB07795,1432,0.0,0.0,2.0,2.0,-3.493732,-1.743561
327180,DB01029,3725,DB08010,1432,0.0,0.0,2.0,2.0,-3.493732,-1.743561
327181,DB01029,3725,DB08015,1432,0.0,0.0,2.0,2.0,-3.493732,-1.743561
327182,DB01029,3725,DB00570,3725,0.0,0.0,0.0,0.0,-3.493732,-3.493732


In [90]:
# remove unwanted rows

drug_targets_separation_filtered = drug_targets_separation_filtered[drug_targets_separation_filtered['drug_a'] != drug_targets_separation_filtered['drug_b']]
drug_targets_separation_filtered       

,drug_a,drug_a_genes,drug_b,drug_b_genes,dA,dB,dAB,sAB,breast_cancer_z_a,breast_cancer_z_b
20,DB00030,3480||5925||3480||5925,DB04147,1072,2.0,0.0,2.0,1.0,-4.091324,-1.126009
21,DB00030,3480||5925||3480||5925,DB08080,57591,2.0,0.0,2.0,1.0,-4.091324,-1.115153
22,DB00030,3480||5925||3480||5925,DB00294,5241||2099||5241||2099,2.0,1.0,1.5,0.0,-4.091324,-7.544839
23,DB00030,3480||5925||3480||5925,DB00304,5241||2099||5241||2099,2.0,1.0,1.5,0.0,-4.091324,-7.544839
24,DB00030,3480||5925||3480||5925,DB00351,5241||2908||5241||2908,2.0,2.0,2.0,0.0,-4.091324,-4.594288
...,...,...,...,...,...,...,...,...,...,...
327178,DB01029,3725,DB07267,1432,0.0,0.0,2.0,2.0,-3.493732,-1.743561
327179,DB01029,3725,DB07795,1432,0.0,0.0,2.0,2.0,-3.493732,-1.743561
327180,DB01029,3725,DB08010,1432,0.0,0.0,2.0,2.0,-3.493732,-1.743561
327181,DB01029,3725,DB08015,1432,0.0,0.0,2.0,2.0,-3.493732,-1.743561


In [91]:
# combine drug_a and drug_b to a new drug column, for later dataframe merging

drug_targets_separation_filtered['drug_a_b'] = drug_targets_separation_filtered[['drug_a', 'drug_b']].apply(lambda x: '_'.join(sorted(x)), axis=1)
drug_targets_separation_filtered



/opt/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,drug_a,drug_a_genes,drug_b,drug_b_genes,dA,dB,dAB,sAB,breast_cancer_z_a,breast_cancer_z_b,drug_a_b
20,DB00030,3480||5925||3480||5925,DB04147,1072,2.0,0.0,2.0,1.0,-4.091324,-1.126009,DB00030_DB04147
21,DB00030,3480||5925||3480||5925,DB08080,57591,2.0,0.0,2.0,1.0,-4.091324,-1.115153,DB00030_DB08080
22,DB00030,3480||5925||3480||5925,DB00294,5241||2099||5241||2099,2.0,1.0,1.5,0.0,-4.091324,-7.544839,DB00030_DB00294
23,DB00030,3480||5925||3480||5925,DB00304,5241||2099||5241||2099,2.0,1.0,1.5,0.0,-4.091324,-7.544839,DB00030_DB00304
24,DB00030,3480||5925||3480||5925,DB00351,5241||2908||5241||2908,2.0,2.0,2.0,0.0,-4.091324,-4.594288,DB00030_DB00351
...,...,...,...,...,...,...,...,...,...,...,...
327178,DB01029,3725,DB07267,1432,0.0,0.0,2.0,2.0,-3.493732,-1.743561,DB01029_DB07267
327179,DB01029,3725,DB07795,1432,0.0,0.0,2.0,2.0,-3.493732,-1.743561,DB01029_DB07795
327180,DB01029,3725,DB08010,1432,0.0,0.0,2.0,2.0,-3.493732,-1.743561,DB01029_DB08010
327181,DB01029,3725,DB08015,1432,0.0,0.0,2.0,2.0,-3.493732,-1.743561,DB01029_DB08015


In [92]:
drug_targets_separation_filtered_unique = drug_targets_separation_filtered.drop_duplicates(subset=['drug_a_b'])
drug_targets_separation_filtered_unique


,drug_a,drug_a_genes,drug_b,drug_b_genes,dA,dB,dAB,sAB,breast_cancer_z_a,breast_cancer_z_b,drug_a_b
20,DB00030,3480||5925||3480||5925,DB04147,1072,2.0,0.0,2.0,1.0,-4.091324,-1.126009,DB00030_DB04147
21,DB00030,3480||5925||3480||5925,DB08080,57591,2.0,0.0,2.0,1.0,-4.091324,-1.115153,DB00030_DB08080
22,DB00030,3480||5925||3480||5925,DB00294,5241||2099||5241||2099,2.0,1.0,1.5,0.0,-4.091324,-7.544839,DB00030_DB00294
23,DB00030,3480||5925||3480||5925,DB00304,5241||2099||5241||2099,2.0,1.0,1.5,0.0,-4.091324,-7.544839,DB00030_DB00304
24,DB00030,3480||5925||3480||5925,DB00351,5241||2908||5241||2908,2.0,2.0,2.0,0.0,-4.091324,-4.594288,DB00030_DB00351
...,...,...,...,...,...,...,...,...,...,...,...
325466,DB08010,1432,DB00570,3725,0.0,0.0,2.0,2.0,-1.743561,-3.493732,DB00570_DB08010
325467,DB08010,1432,DB01029,3725,0.0,0.0,2.0,2.0,-1.743561,-3.493732,DB01029_DB08010
326038,DB08015,1432,DB00570,3725,0.0,0.0,2.0,2.0,-1.743561,-3.493732,DB00570_DB08015
326039,DB08015,1432,DB01029,3725,0.0,0.0,2.0,2.0,-1.743561,-3.493732,DB01029_DB08015


# compare drug_combs

In [94]:
# read the her2 breast cancer gene related drug combiantion from cdcdb

her2_bc_drug_combs = pd.read_excel('/Users/yanlixu/Desktop/pingan/合作/CDK12/network_based/3_drug_combination_results.xlsx', sheet_name= 'all_her2_bc_genes_drug_combs')
her2_bc_drug_combs



,her2_drug,index,drugs,drugbank_identifiers,pubchem_identifiers,source_id,source
0,DB00257,665,"[[""nystatin""], [""sirolimus""], [""alemtuzumab""],...","[""DB00646"", ""DB00877"", ""DB00087"", ""DB00787"", ""...","[""CID44424838"", ""-1"", ""-1"", ""CID2022"", ""CID674...",NCT00078559,clinicaltrials.gov
1,DB00257,18248,"[[""Antifungal Agents"", ""clotrimazole""], [""Anti...","[""DB00257"", ""DB01190""]","[""CID2812"", ""CID446598""]",NCT02860845,clinicaltrials.gov
2,DB00257,30061,"['clotrimazole', 'betamethasone dipropionate']","['DB00257', 'DB00443']","['CID2812', 'CID21800']",US20170290810,patents
3,DB00257,31259,"['metronidazole', 'colchicine', 'clotrimazole'...","['DB00916', 'DB01394', 'DB00257', 'DB00787', '...","['CID4173', 'CID23890', 'CID2812', 'CID2022', ...",CA2612353,patents
4,DB00257,31382,"['metronidazole', 'clotrimazole', 'chlorhexidi...","['DB00916', 'DB00257', 'DB00878']","['CID4173', 'CID2812', 'CID9571016']",CN101890003,patents
...,...,...,...,...,...,...,...
17469,DB02375,39175,"['quercetin', 'apigenin', 'baicalein', 'luteol...","['DB04216', 'DB07352', '-1', 'DB15584', 'DB018...","['CID5280343', 'CID5280443', 'CID5281605', 'CI...",US20190314375,patents
17470,DB02375,39202,"['myricetol', 'hesperetin']","['DB02375', 'DB01094']","['CID5281672', 'CID72281']",US20190365703,patents
17471,DB02375,42360,"['piperine', 'gallic acid', 'myricetol', 'hesp...","['DB12582', '-1', 'DB02375', 'DB01094']","['CID638024', 'CID370', 'CID5281672', 'CID72281']",WO2019045677,patents
17472,DB01108,34394,"['trilostane', 'metyrapone', 'fadrozole', 'ket...","['DB01108', 'DB01011', '-1', 'DB01026', 'DB008...","['CID656583', 'CID27284', 'CID59693', 'CID3823...",EP1886695,patents


In [95]:
# remove rows which don't have 2 drugs
her2_bc_drug_combs = her2_bc_drug_combs.astype(str)
her2_bc_drug_combs['num_of_drug'] = her2_bc_drug_combs['drugbank_identifiers'].apply(eval).apply(len)
her2_bc_drug_combs_2_drug = her2_bc_drug_combs[her2_bc_drug_combs['num_of_drug']==2]
her2_bc_drug_combs_2_drug



,her2_drug,index,drugs,drugbank_identifiers,pubchem_identifiers,source_id,source,num_of_drug
1,DB00257,18248,"[[""Antifungal Agents"", ""clotrimazole""], [""Anti...","[""DB00257"", ""DB01190""]","[""CID2812"", ""CID446598""]",NCT02860845,clinicaltrials.gov,2
2,DB00257,30061,"['clotrimazole', 'betamethasone dipropionate']","['DB00257', 'DB00443']","['CID2812', 'CID21800']",US20170290810,patents,2
5,DB00257,33644,"['clotrimazole', 'timosaponin b-ii']","['DB00257', '-1']","['CID2812', '-1']",CN109431968,patents,2
6,DB00257,33645,"['clotrimazole', 'ophiopogonin b']","['DB00257', '-1']","['CID2812', 'CID46173857']",CN109431969,patents,2
7,DB00257,35178,"['clotrimazole', 'diclofenac sodium']","['DB00257', 'DB00586']","['CID2812', 'CID5018304']",EP3482744,patents,2
...,...,...,...,...,...,...,...,...
17458,DB02375,34538,"['quercetin', 'myricetol']","['DB04216', 'DB02375']","['CID5280343', 'CID5281672']",EP2129371,patents,2
17462,DB02375,36832,"['trans-resveratrol', 'myricetol']","['DB02709', 'DB02375']","['CID445154', 'CID5281672']",KR2016140482,patents,2
17464,DB02375,37904,"['quercetin', 'myricetol']","['DB04216', 'DB02375']","['CID5280343', 'CID5281672']",US20120252887,patents,2
17470,DB02375,39202,"['myricetol', 'hesperetin']","['DB02375', 'DB01094']","['CID5281672', 'CID72281']",US20190365703,patents,2


In [97]:
# convert drugbank_identifiers to a new column for later merging
her2_bc_drug_combs_2_drug['two_drug'] = her2_bc_drug_combs_2_drug['drugbank_identifiers'].apply(eval).apply(sorted).apply(lambda x: '_'.join(x))
her2_bc_drug_combs_2_drug


/opt/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,her2_drug,index,drugs,drugbank_identifiers,pubchem_identifiers,source_id,source,num_of_drug,two_drug
1,DB00257,18248,"[[""Antifungal Agents"", ""clotrimazole""], [""Anti...","[""DB00257"", ""DB01190""]","[""CID2812"", ""CID446598""]",NCT02860845,clinicaltrials.gov,2,DB00257_DB01190
2,DB00257,30061,"['clotrimazole', 'betamethasone dipropionate']","['DB00257', 'DB00443']","['CID2812', 'CID21800']",US20170290810,patents,2,DB00257_DB00443
5,DB00257,33644,"['clotrimazole', 'timosaponin b-ii']","['DB00257', '-1']","['CID2812', '-1']",CN109431968,patents,2,-1_DB00257
6,DB00257,33645,"['clotrimazole', 'ophiopogonin b']","['DB00257', '-1']","['CID2812', 'CID46173857']",CN109431969,patents,2,-1_DB00257
7,DB00257,35178,"['clotrimazole', 'diclofenac sodium']","['DB00257', 'DB00586']","['CID2812', 'CID5018304']",EP3482744,patents,2,DB00257_DB00586
...,...,...,...,...,...,...,...,...,...
17458,DB02375,34538,"['quercetin', 'myricetol']","['DB04216', 'DB02375']","['CID5280343', 'CID5281672']",EP2129371,patents,2,DB02375_DB04216
17462,DB02375,36832,"['trans-resveratrol', 'myricetol']","['DB02709', 'DB02375']","['CID445154', 'CID5281672']",KR2016140482,patents,2,DB02375_DB02709
17464,DB02375,37904,"['quercetin', 'myricetol']","['DB04216', 'DB02375']","['CID5280343', 'CID5281672']",US20120252887,patents,2,DB02375_DB04216
17470,DB02375,39202,"['myricetol', 'hesperetin']","['DB02375', 'DB01094']","['CID5281672', 'CID72281']",US20190365703,patents,2,DB01094_DB02375


In [103]:
merged_df_1 = pd.merge(drug_targets_separation_filtered_unique, her2_bc_drug_combs_2_drug, left_on="drug_a_b", right_on="two_drug")
merged_df_1 = merged_df_1.drop(['her2_drug', 'index'], axis=1)
merged_df_1 = merged_df_1.drop_duplicates()
merged_df_1




,drug_a,drug_a_genes,drug_b,drug_b_genes,dA,dB,dAB,sAB,breast_cancer_z_a,breast_cancer_z_b,drug_a_b,drugs,drugbank_identifiers,pubchem_identifiers,source_id,source,num_of_drug,two_drug
0,DB00030,3480||5925||3480||5925,DB01252,5468,2.0,0.0,1.333333,0.333333,-4.091324,-2.227916,DB00030_DB01252,"[[""KAD 1229""], [""Insulin""]]","[""DB01252"", ""DB00030""]","[""CID121891"", ""-1""]",NCT02154347,clinicaltrials.gov,2,DB00030_DB01252
2,DB00030,3480||5925||3480||5925,DB02701,142,2.0,0.0,2.000000,1.000000,-4.091324,-2.289581,DB00030_DB02701,"['niacinamide', 'human insulin']","['DB02701', 'DB00030']","['CID936', 'CID118984375']",CN103249427,patents,2,DB00030_DB02701
3,DB00030,3480||5925||3480||5925,DB02701,142,2.0,0.0,2.000000,1.000000,-4.091324,-2.289581,DB00030_DB02701,"['niacinamide', 'human insulin']","['DB02701', 'DB00030']","['CID936', 'CID118984375']",WO2012080362,patents,2,DB00030_DB02701
6,DB00046,3480,DB00047,3480,0.0,0.0,0.000000,0.000000,-0.963609,-0.963609,DB00046_DB00047,"[[""insulin glargine""], [""insulin lispro"", ""Hum...","[""DB00047"", ""DB00046""]","[""CID118984454"", ""CID16132438""]",NCT01215955,clinicaltrials.gov,2,DB00046_DB00047
8,DB00046,3480,DB00047,3480,0.0,0.0,0.000000,0.000000,-0.963609,-0.963609,DB00046_DB00047,"[[""insulin glargine""], [""insulin lispro"", ""Hum...","[""DB00047"", ""DB00046""]","[""CID118984454"", ""CID16132438""]",NCT01454284,clinicaltrials.gov,2,DB00046_DB00047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711,DB02300,7421,DB02701,142,0.0,0.0,2.000000,2.000000,-2.606916,-2.289581,DB02300_DB02701,"['niacinamide', 'calcipotriol']","['DB02701', 'DB02300']","['CID936', '-1']",WO2012049677,patents,2,DB02300_DB02701
1714,DB04942,5914,DB00307,5914||6256||6258||5914||6256||6258||5914||6256...,0.0,1.0,0.500000,0.000000,-2.369024,-2.627013,DB00307_DB04942,"['tamibarotene', 'bexarotene']","['DB04942', 'DB00307']","['CID108143', 'CID82146']",CN106456770,patents,2,DB00307_DB04942
1716,DB00374,5467,DB00482,1432,0.0,0.0,2.000000,2.000000,-0.152169,-1.743561,DB00374_DB00482,"['treprostinil sodium', 'celecoxib']","['DB00374', 'DB00482']","['CID23663413', 'CID2662']",US20120010159,patents,2,DB00374_DB00482
1718,DB00482,1432,DB01628,1432,0.0,0.0,0.000000,0.000000,-1.743561,-1.743561,DB00482_DB01628,"[[""etoricoxib""], [""celecoxib""]]","[""DB01628"", ""DB00482""]","[""CID123619"", ""CID2662""]",NCT01017380,clinicaltrials.gov,2,DB00482_DB01628


In [104]:
merged_df_1.to_csv('her2_2drug_combs.csv')

In [ ]:
network_file = '/Users/yanlixu/Desktop/git_code/drug_repurposing_202206/network_based_2/toolbox_simplified/ppi_chenfeixiong.tsv'

drug_gene_file = '/Users/yanlixu/Desktop/pingan/合作/CDK12/network_based/1_her2_breast_cancer_20220620.xlsx'

# read network
G = read_network(network_file)
# get all genes ad remove self links
all_genes_in_network = set(G.nodes())
remove_self_links(G)

# read data from excel
drug_genes = pd.read_excel(drug_gene_file, sheet_name='5_drug_target_w_diamond_100_cmp')
drug_genes = drug_genes.astype(str)
# convert genes column to list
drug_genes['Drug_Target (Gene Eentrez IDs)'] = drug_genes['Drug_Target (Gene Eentrez IDs)'].apply(lambda x: x[1:-1].split('||'))
# dataframe to dictionary
drug_genes.set_index('DrugID (DrugBank ID)', inplace=True)
drug_gene_dic = drug_genes.to_dict()['Drug_Target (Gene Eentrez IDs)']

fout = open('her2_drug_genes_separation.txt', 'w')
fout.write('drug_a' + '\t' + 'drug_a_genes' + '\t' + 'drug_b' + '\t' + 'drug_b_genes' + '\t' + 'dA' + '\t' + 'dB' + '\t' + 'dAB' + '\t' + 'sAB' + '\n')
for drug_a in drug_gene_dic.keys():
    print(drug_a)
    drug_a_genes = drug_gene_dic[drug_a]
    for drug_b in drug_gene_dic.keys():
        drug_b_genes = drug_gene_dic[drug_b]
        result = calculate_distance_bw_2gene_sets(G, all_genes_in_network, drug_a_genes, drug_b_genes)
        fout.write(drug_a + '\t' + '||'.join(drug_gene_dic[drug_a]) + '\t' + drug_b + '\t' + '||'.join(drug_gene_dic[drug_b])+ '\t' + str(result[0]) + '\t' + str(result[1]) + '\t' + str(result[2]) + '\t' + str(result[3]) + '\n')
fout.close()
